In [21]:
import os, sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import database
from snowballing.operations import reload, work_by_varname, load_work_map_all_years
from snowballing.snowballing import snowballing, create_provenance, log_to_provn
reload()


In [37]:
def busca(*libraries, seed=None):
    if seed is not None:
        iterable = ((1, work_by_varname(x)) for x in seed)
    else:
        iterable = load_work_map_all_years()
    seedset = set()
    for _, work in iterable:
        for library in libraries:
            if int(getattr(work, library, 0)):
                seedset.add(work)
                break
    return seedset
    
#busca("acm", seed=["wohlin2014a", "briand2000a"])
busca("acm")

{A systematic mapping study on technical debt and its management,
 Guidelines for Snowballing in Systematic Literature Studies and a Replication in Software Engineering}

In [39]:
seedset = busca("acm", "scopus")
acm = sum(int(getattr(x, 'acm', 0)) for x in seedset)
acm

2

In [27]:
from estrategias import padrao as estrategia
#from estrategias import aplica_par_backward_forward as estrategia
#from estrategias import todo_backward_e_depois_todo_forward as estrategia

reload()
frontier = {x for x in seedset}

filter_function = lambda x: x.category in ("snowball", "forward", "backward")
frontier, log, visited = estrategia(
    frontier, filter_function
)

log

[Step(name='start', new_references=3, new_related=3, total_visited=3, total_related=3),
 Step(name='backward', new_references=21, new_related=1, total_visited=24, total_related=4),
 Step(name='forward', new_references=41, new_related=9, total_visited=65, total_related=13),
 Step(name='backward', new_references=0, new_related=0, total_visited=65, total_related=13),
 Step(name='forward', new_references=0, new_related=0, total_visited=65, total_related=13)]

In [28]:
frontier

{A replicated assessment and comparison of common software cost modeling techniques,
 A first systematic mapping study on combinatorial interaction testing for software product lines,
 Modelling language quality evaluation in model-driven information systems engineering: A roadmap,
 Guidelines for conducting systematic mapping studies in software engineering: An update,
 Guidelines for Snowballing in Systematic Literature Studies and a Replication in Software Engineering,
 Automated testing of mobile applications: a systematic map and review,
 A review of code smell mining techniques,
 A systematic mapping study of search-based software engineering for software product lines,
 Software engineering knowledge areas in startup companies: a mapping study,
 Multiphysics and multiscale software frameworks: an annotated bibliography,
 Experiences from using snowballing and database searches in systematic literature studies,
 Evidence-based software engineering,
 A systematic mapping study on 

In [4]:
with open("output/snowballing.provn", "w") as provn:
    provn.write(log_to_provn(log))

In [11]:
!provconvert.bat -infile snowballing.provn -outfile snowballing.dot

'provconvert.bat' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
with open("output/snowballing.dot", "r") as prov_dot:
    text = prov_dot.read().replace('rankdir="BT";', 'rankdir="RL"; ranksep="0.1";')
with open("output/snowballing.dot", "w") as prov_dot:
    prov_dot.write(text)

FileNotFoundError: [Errno 2] No such file or directory: 'output/snowballing.dot'

In [13]:
!dot -Tpng output/snowballing.dot -ooutput/snowballing.png
!dot -Tsvg output/snowballing.dot -ooutput/snowballing.svg
!inkscape -D -z --file=output/snowballing.svg --export-pdf=../csur/figs/snowballing.pdf

'dot' is not recognized as an internal or external command,
operable program or batch file.
'dot' is not recognized as an internal or external command,
operable program or batch file.
'inkscape' is not recognized as an internal or external command,
operable program or batch file.


In [14]:
len(visited)

64

In [15]:
len([x for x in visited if x.category in {"snowball"}])

1

In [16]:
from snowballing.approaches import get_approaches
len(get_approaches())

0

In [18]:
len([(x, x.year) for x in visited
     if x.category in {"nofile"}
     if x.place.name != "Tech Report"
     if x.place.name != "Book"
     if x.place.name != "Patent"
     if x.place.name == "Conference"
     if x.display != x.place.acronym
    ])
#count = 0
#for x in visited:
#     if hasattr(x, 'dglibrary'):
#        print(x.dglibrary)
#        count = 1 + count
#print (count)
len([
    (x, x.year) for x in visited
     if hasattr(x, 'dglibrary') and x.dglibrary=="Scopus"
    
])

def count_ref(library,obj):
    len_dglibrary = len([(x, x.year) for x in obj
                             if hasattr(x, 'dglibrary') and library in x.dglibrary  
                        ])
    len_backward = len([(x, x.year) for x in obj
                             if x.category=="backward"  
                        ])
    len_forward = len([(x, x.year) for x in obj
                             if x.category=="forward"  
                        ])
    qtdref=0
    qtdcitations=0
    for x in obj:
        qtdref=qtdref + int(getattr(x, "references", 0))
        
    scopus = sum(int(getattr(x, "scopus", 0)) for x in obj)
            
    for x in obj:
        qtdcitations=qtdcitations + int(getattr(x, "citations", 0))
    return len_dglibrary,qtdref,qtdcitations,len_backward,len_forward
     

print ("Digital Library, Referencias, Citações, Qtd Backward, Qtd Forward")
print(count_ref("Scopus,ACM",visited))

log

print (log[0].name, log[0].new_references)
print(log[1].name, log[1].new_references)
print(log[3].name, log[3].new_references)
print("visitados", len(visited))

Digital Library, Referencias, Citações, Qtd Backward, Qtd Forward
(1, 52, 400, 1, 10)
start 1
backward 21
forward 42
visitados 64


In [ ]:
len([x for x in visited if x.category in {"lang"}])

In [ ]:
len([x for x in visited if x.category in {"site"}])

In [ ]:
len([x for x in visited
     if x.place.name == "Tech Report"
    ])

In [ ]:
len([x for x in visited
     if x.place.name == "ESEM"
    ])

In [ ]:
{x.category for x in visited}

In [ ]:
[(x.year, x.authors, x.name, x.place.name, x.request) for x in visited
 if x.category in {"nofile"}
 if x.place.name != "Tech Report"
 if x.place.name != "Book"
 if x.place.name != "Patent"
 if x.display != x.place.acronym]

In [ ]:
len(_)